In [ ]:
import torch

### Comparison

#### CNN
+ 权重共享 平移不变性、可并行计算
+ 滑动窗口 局部关联性建模、依靠多层堆积来进行长程建模
+ 对相对位置敏感、对绝对位置不敏感

#### RNN 依次有序递归建模
+ 对顺序敏感
+ 串行计算耗时
+ 长程建模能力弱
+ 计算复杂度与序列长度呈线性关系
+ 单步计算复杂度不变
+ 对相对位置敏感，对绝对位置敏感
  
#### tranformer
+ 无局部假设
  + 可并行计算
  + 对相对位置不敏感
+ 无有序假设
  + 需要位置编码反映位置变化对特征的影响
  + 对绝对位置不敏感
+ 任意两字符都可以建模
  + 擅长长短程建模
  + 自注意力机制需要序列长度的平方级复杂运算

### Transformer encoder
+ input word embedding
  由稀疏的one-hot进入一个不带bias的FFN得到一个稠密的连续箱梁
+ position encoding
  + 通过sin/cos来固定表征
    + 每个位置确定
    + 对于不同的句子，相同位置的距离一致
    + 可以推广到更长的测试句子
  + pe(pos+k)可以写成pe(k)的线性组合
  + 通过残差连接来是的位置信息流入深层
+ multi-head self-attention
  + 建模能力更强，表征空间更丰富
  + 由多组Q\K\V构成，每组单独计算一个attention向量
  + 把每组的attention向量拼起来，并进入一个FFN中得到最终的向量
+ feed-forward network
  + 只考虑每个单独位置进行建模
  + 不同位置参数共享
  + 类似于1*1 pointwise convolution

### Transformer Decoder
+ output word embedding
+ masked multi-head self-attention
+ multi-head cross-attention
+ feed-forward network
+ softmax

In [ ]:
from torch import Tensor
from .. import functional as F